# Project imports

In [34]:
"""
All needed imports included here
"""
%load_ext autoreload
%autoreload 2
from pathlib import Path
import numpy as np
import matplotlib as plt
import torch
import pytorch_lightning as pl


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cuda


# Data Loading step

In [35]:
"""
Create data loaders and augmentations needed here
"""
from Data.ShapeNetDataLoader import ShapeNetVoxelData
from Utils.visualization import visualize_occupancy

overfit = True

shapenet_core_path = Path("D:\ShapeNetCore.v2")
shapenet_splits_csv_path = Path("Data/shapenet_splits.csv")
train_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="train", overfit=overfit)
val_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="val", overfit= overfit)
test_data = ShapeNetVoxelData(shapenet_core_path=shapenet_core_path, shapenet_splits_csv_path=shapenet_splits_csv_path, split="test", overfit=overfit)

train_sample = train_data[7]
print(f'Voxel Dimensions: {train_sample.shape}')
print(f"Test Set Size: {len(train_data)}")

print(train_sample)
visualize_occupancy(train_sample.squeeze(), flip_axes=True)

Voxel Dimensions: (1, 128, 128, 128)
Test Set Size: 16
[[[[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  ...

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   ...
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]]

  [[0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
   [0. 0. 0. ... 0. 0. 0.]
  

Output()

# Reconstruction Networks

In [10]:
"""
creation, training, and testing of the image2mesh reconstruction networks
"""
from Networks.Image2Mesh import Image2Voxel

model = Image2Voxel(dataloader,dataloader,dataloader,device)

X = torch.rand(10,3,32,32)
pred = model(X)

NameError: name 'dataloader' is not defined

In [7]:
trainer = pl.Trainer(
    max_epochs=15,
    gpus=1 if torch.cuda.is_available() else None,
    profiler=True,
    progress_bar_refresh_rate=15
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | Network | 67.1 M


Epoch 0:   0%|          | 0/4 [00:00<?, ?it/s]                        

F:\Python\Anaconda\envs\ml3d\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


Epoch 14: 100%|██████████| 4/4 [00:13<00:00,  3.46s/it, loss=0.483, v_num=16]



Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_fit_start        	|  0.0            	|  0.0            
on_validation_start 	|  0.0019375      	|  0.031          
on_validation_epoch_start	|  0.0            	|  0.0            
on_validation_batch_start	|  0.0            	|  0.0            
validation_step_end 	|  0.0            	|  0.0            
on_validation_batch_end	|  0.0            	|  0.0            
on_validation_epoch_end	|  0.0            	|  0.0            
on_validation_end   	|  5.8144         	|  93.03          
on_train_start      	|  0.016          	|  0.016          
on_epoch_start      	|  0.0010667      	|  0.016          
on_train_epoch_start	|  0.0            	|  0.0            
get_train_batch     	|  0.0037         	|  0.111          
on_batch_start      	|  0.0            	|  0.0            
on_train_batch_start	|  0.0            	|  0.0            
training_ste

1

# Purifying predicted Meshes

In [ ]:
"""
Code to purify meshes predicted by the previous networks to be used in the retrieval step
"""

# Mesh Encoding

In [77]:
"""
AutoEncoder Models and/or different techniques used to encode the mesh to a smaller dimensions
"""
from Networks.VoxelAutoencoder import VoxelAutoencoder

dim = 128
model = VoxelAutoencoder(dim, train_data, val_data, test_data, device)
x = torch.randn(3, 1, dim, dim, dim)
print(x)

tensor([[[[[-2.7178e+00,  8.0033e-01,  6.6767e-02,  ..., -1.3776e+00,
            -6.7513e-01, -8.7225e-01],
           [ 3.9983e-01,  1.5722e-01,  6.8232e-01,  ..., -6.8808e-01,
             2.2576e-01, -8.1396e-01],
           [ 1.3829e+00, -1.6483e+00, -3.3922e-01,  ..., -1.4518e-01,
            -1.3805e+00,  1.9384e-01],
           ...,
           [-1.1278e+00,  1.2020e-02, -7.9293e-01,  ..., -3.5333e-01,
             1.1964e+00,  6.3626e-02],
           [-1.3855e+00,  2.6979e+00, -8.4345e-02,  ...,  1.3639e+00,
             9.2152e-01,  6.5392e-01],
           [ 1.2381e+00,  1.0096e+00,  5.2653e-01,  ..., -2.4872e-01,
            -2.1487e-01,  2.7373e-01]],

          [[-4.9731e-01,  9.4383e-01,  1.1118e-02,  ..., -4.3587e-01,
             4.7055e-01,  1.3597e-01],
           [ 4.4655e-01, -1.6423e+00,  1.7461e+00,  ..., -2.7654e-01,
            -9.7211e-01,  8.4967e-01],
           [ 1.0482e+00, -5.4974e-01, -1.2931e+00,  ..., -1.2475e+00,
            -1.3721e+00, -1.3244e+00],
 

In [78]:
trainer = pl.Trainer(
    max_epochs=64,
    gpus=1 if torch.cuda.is_available() else None,
    progress_bar_refresh_rate=2, 
)

trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | Network | 8.7 M 
----------------------------------
8.7 M     Trainable params
0         Non-trainable params
8.7 M     Total params
34.956    Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]tensor([[[[[0.4591, 0.4625, 0.4071,  ..., 0.4625, 0.4071, 0.4647],
           [0.5294, 0.4968, 0.5057,  ..., 0.4968, 0.5057, 0.5050],
           [0.4601, 0.4232, 0.3754,  ..., 0.4232, 0.3755, 0.4224],
           ...,
           [0.5294, 0.4967, 0.5057,  ..., 0.4968, 0.5057, 0.5050],
           [0.4602, 0.4232, 0.3755,  ..., 0.4232, 0.3755, 0.4223],
           [0.5078, 0.4932, 0.5295,  ..., 0.4932, 0.5296, 0.4677]],

          [[0.5487, 0.4266, 0.5534,  ..., 0.4266, 0.5534, 0.4769],
           [0.5015, 0.4352, 0.4912,  ..., 0.4352, 0.4912, 0.4510],
           [0.4894, 0.3412, 0.4697,  ..., 0.3411, 0.4696, 0.4131],
           ...,
           [0.5015, 0.4352, 0.4912,  ..., 0.4352, 0.4912, 0.4510],
           [0.4893, 0.3413, 0.4697,  ..., 0.3412, 0.4697, 0.4130],
           [0.4348, 0.4345, 0.4630,  ..., 0.4345, 0.4630, 0.4585]],

          [[0.4717, 0.4338, 0.4013,  ..., 0.4339, 0.4013, 0.4447],
           [0.5251, 0.5348, 0.4

In [79]:
train_test_sample = train_data[0]
print(np.sum(train_test_sample))
visualize_occupancy(train_test_sample.squeeze(), flip_axes=True)

tensor_test = torch.from_numpy(train_test_sample[np.newaxis, :])
print(tensor_test.shape)

model.eval()
with torch.no_grad():
    decoded_test = model(tensor_test)

print(decoded_test)
tmp_decoded = decoded_test.clone()
tmp_decoded[decoded_test<0.5] = 0
tmp_decoded[decoded_test>=0.5] = 1

decoded_test_np = tmp_decoded.squeeze().detach().numpy()
# TODO: convert to occupancy grid

visualize_occupancy(decoded_test_np, flip_axes=True)
print(np.sum(decoded_test_np))
#visualize_occupancy(np.ones((32, 32, 32)), flip_axes=True)

15056.0


Output()

torch.Size([1, 1, 128, 128, 128])
tensor([[[[[1.7492e-01, 6.5111e-02, 4.3822e-02,  ..., 6.5111e-02,
            4.3822e-02, 1.8074e-01],
           [8.8113e-02, 6.1193e-03, 6.9680e-03,  ..., 6.2053e-03,
            7.5857e-03, 7.2277e-02],
           [6.5087e-02, 5.3783e-03, 3.8602e-03,  ..., 5.7717e-03,
            4.4425e-03, 4.8294e-02],
           ...,
           [8.8113e-02, 6.1193e-03, 6.9680e-03,  ..., 7.2079e-03,
            1.0500e-02, 7.8801e-02],
           [6.5087e-02, 5.3783e-03, 3.8602e-03,  ..., 6.2655e-03,
            5.2625e-03, 5.0561e-02],
           [2.1944e-01, 6.6437e-02, 8.9180e-02,  ..., 7.2190e-02,
            1.2080e-01, 1.7911e-01]],

          [[1.0390e-01, 5.6839e-03, 1.0710e-02,  ..., 8.2103e-03,
            1.3401e-02, 8.6149e-02],
           [7.1859e-03, 5.6023e-05, 5.7307e-05,  ..., 7.3584e-05,
            8.5727e-05, 6.1815e-03],
           [6.6127e-03, 3.5286e-05, 4.9842e-05,  ..., 4.7147e-05,
            7.2922e-05, 5.1806e-03],
           ...,
     

Output()

0.0


# Mesh Retreival Networks

In [ ]:
"""
Models/Techniques to use the previous encoding steps to retreive objects from a specified database
"""

# Inference and Full Testing

In [ ]:
"""
Testing the entire pipeline implemented with added visualizations and discussions.
"""

# Citations

[1].....